In [1]:
import pandas as pd
import numpy as np

from bokeh.palettes import Paired as palette
from bokeh.plotting import figure, output_notebook, output_file, show, save
from bokeh.models.widgets import Panel, Tabs

output_notebook()

Loading BokehJS ...

In [2]:
output = pd.read_csv("../../Data/output.csv", index_col=0, parse_dates=True)["2006-02":]
salinity = pd.read_csv("../../Data/salinity.csv", index_col=0, parse_dates=True)

together = pd.concat([output, salinity], axis=1)

In [3]:
def minmax(X):
    return (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

for col in together.columns:
    together[col] = minmax(together[col].values)

In [4]:
p1 = figure(title="Volume of the Small Aral Sea", x_axis_type="datetime", 
                plot_width=900, plot_height=600)

p1.grid.grid_line_alpha=0.5
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'Volume, km³'

# simulations
#p1.patch(np.append(inflow_sim.index, inflow_sim.index[::-1]), 
#         np.append(inflow_sim["Min"].values, inflow_sim["Max"].values[::-1]), 
#         color='dimgray', fill_alpha=0.2, legend='Simulations spread')

p1.line(output.index, output["Vol_sim"], color='red', legend='Simulated Volume')
p1.circle(output.index, output["Vol_sim"], color='red', fill_color="red", size=4, legend='Simulated Volume')

p1.line(output.index, output["Vol_obs"], color='blue', legend='Observed Volume')
p1.circle(output.index, output["Vol_obs"], color='blue', fill_color="blue", size=6, legend='Observed Volume')

tab1 = Panel(child=p1, title="Volume")

# forecasts
#p1.patch(np.append(forecast.index, forecast.index[::-1]), 
#         np.append(forecast["Qfrc_min"].values, forecast["Qfrc_max"].values[::-1]), 
#         color='royalblue', fill_alpha=0.2, legend='Forecast spread')
#p1.line(forecast.index, forecast["Qfrc_mean"], color='royalblue', legend='Forecast mean')
#p1.circle(forecast.index, forecast["Qfrc_mean"], color='royalblue', fill_color="dodgerblue", size=8, legend='Forecast mean')  

############################################
p2 = figure(title="Inflow/Outflow dynamics of the Small Aral Sea", x_axis_type="datetime", 
                plot_width=900, plot_height=600)

p2.grid.grid_line_alpha=0.5
p2.xaxis.axis_label = 'Date'
p2.yaxis.axis_label = 'Volume, km³'

# observations
p2.line(output.index, output["Inflow"], color='blue', legend='Simulated Inflow')
p2.circle(output.index, output["Inflow"], color='blue', fill_color="blue", size=4, legend='Simulated Inflow')

p2.line(output.index, output["Outflow"], color='green', legend='Simulated Outflow')
p2.circle(output.index, output["Outflow"], color='green', fill_color="green", size=4, legend='Simulated Outflow')
#p2.circle(inflow_obs.index, inflow_obs["Obs"], color='dimgray', fill_color="dimgray", size=8, legend='Observations')

tab2 = Panel(child=p2, title="Inflow/Outflow")

#############################################
p3 = figure(title="Meteorological inputs for the Small Aral Sea Water Balance Model", x_axis_type="datetime", 
                plot_width=900, plot_height=600)

p3.grid.grid_line_alpha=0.5
p3.xaxis.axis_label = 'Date'
p3.yaxis.axis_label = 'Volume, km³'

p3.vbar(x=output.index, top=output["Prec"], width=0.9*(1000*60*60*24*30), color="blue", alpha=0.5, legend="Precipitation")
p3.vbar(x=output.index, top=output["Evap"], width=0.9*(1000*60*60*24*30), color="green", alpha=0.5, legend="Evaporation")

tab3 = Panel(child=p3, title="Meteo")

#############################################
p4 = figure(title="Salinity dynamics of the Small Aral Sea", x_axis_type="datetime", 
                plot_width=900, plot_height=600)

p4.grid.grid_line_alpha=0.5
p4.xaxis.axis_label = 'Date'
p4.yaxis.axis_label = 'Salinity, ‰'

# observations
p4.line(salinity.index, salinity["Sal"], color='red', legend='Simulated Salinity')
p4.circle(salinity.index, salinity["Sal"], color='red', fill_color="red", size=4, legend='Simulated Salinity')
#p2.circle(inflow_obs.index, inflow_obs["Obs"], color='dimgray', fill_color="dimgray", size=8, legend='Observations')

tab4 = Panel(child=p4, title="Salinity")

#############################################
p5 = figure(title="Dynamics of the Small Aral Sea Water Balance Components (normalized range [0,1]). Click on the legend!", 
            x_axis_type="datetime", plot_width=900, plot_height=600)

p5.grid.grid_line_alpha=0.5
p5.xaxis.axis_label = 'Date'
p5.yaxis.axis_label = 'Normalized values of Water Balance Components, dimensionless'

for col, col_name, color in zip(["Inflow", "Outflow", "Prec", "Evap", "Vol_obs", "Vol_sim", "Sal"], 
                                ["Inflow", "Outflow", "Precipitation", "Evaporation", "Simulated Volume", "Observed Volume", "Salinity"],
                                palette[7]):
    p5.line(together.index, together[col], line_width=2, color=color, alpha=0.8, legend=col_name)

p5.legend.location = "top_left"
p5.legend.click_policy="hide"

tab5 = Panel(child=p5, title="All-in")

#############################################
tabs = Tabs(tabs=[ tab1, tab4, tab3, tab2, tab5 ])

show(tabs)

In [5]:
output_file("../docs/app.html", title="SMASHI Water Balance App")
save(tabs)

'/home/georgy/Documents/Aral/WB/docs/app.html'